# Imports and Colab Mount

In [1]:
import datetime
import seaborn as sn
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.stats as stats
import glob
from math import sqrt
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, LSTM, LeakyReLU, Dropout, GRU, SimpleRNN, Input, LSTM, Dense, Bidirectional, Concatenate, Reshape, Lambda, Bidirectional
from keras.models import Model, Sequential
from keras import backend as K
from tensorflow.keras import layers
from keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
import seaborn as sns

from numpy.random import seed
#from tensorflow import set_random_seed

%matplotlib inline

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# Load Data

In [3]:
hourly = pd.read_excel("/content/drive/MyDrive/Hybrid Transformer/Datasets/Vento/BASERECIFEhorario.xlsx", index_col=0, parse_dates=[["Data", "HORA (UTC)"]])
hourly = hourly[ ['Velocidade'] + [ col for col in hourly.columns if col != 'Velocidade' ] ]
quarto = int(hourly["Velocidade"].shape[0]/4)

train_h = hourly.iloc[:quarto*2]
valid_h = hourly.iloc[quarto*2:quarto*3]
test_h = hourly.iloc[quarto*3:]

resid_train = pd.read_csv("/content/drive/MyDrive/Hybrid Transformer/Datasets/Vento/recifehora_resid_train.csv").drop("Unnamed: 0", axis=1)
resid_test = pd.read_csv("/content/drive/MyDrive/Hybrid Transformer/Datasets/Vento/recifemensal_resid_test.csv").drop("Unnamed: 0", axis=1)

sarima_pred = pd.read_csv("/content/drive/MyDrive/Hybrid Transformer/Datasets/Vento/recifehora_prediction.csv").drop("Unnamed: 0", axis=1)
sarima_pred2 = pd.read_csv("/content/drive/MyDrive/Hybrid Transformer/Datasets/Vento/recifemensal_prediction2.csv").drop("Unnamed: 0", axis=1)

In [4]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [5]:
def make_data(data, timestep, resid_check=False):
  values = data.values
  values = values.astype('float32')
  #scaler = MinMaxScaler(feature_range=(0, 1))
  #scaled = scaler.fit_transform(values)
  
  timestep = timestep
  n_features = data.shape[1]
  n_obs = timestep * n_features
  reframed = series_to_supervised(values, timestep, 1)
  reframed = reframed.iloc[: , :1-n_features]

  values = reframed.values
  indice1 = train_h.shape[0]
  indice2 = valid_h.shape[0]

  train = values[:indice1, :]
  valid = values[indice1:indice1+indice2, :]
  test = values[indice1+indice2:, :]

  train_X, train_y = train[:, :-1], train[:, -1]
  valid_X, valid_y = valid[:, :-1], valid[:, -1]
  test_X, test_y = test[:, :-1], test[:, -1]

  if (resid_check==True):
    train_y = resid_train.values[:quarto*2]
    valid_y = resid_train.values[quarto*2:]

  scaler = MinMaxScaler(feature_range=(0, 1)).fit(train_X)
  train_X = scaler.transform(train_X)
  valid_X = scaler.transform(valid_X)
  test_X = scaler.transform(test_X)

  scaler_y = MinMaxScaler(feature_range=(0, 1)).fit(train_y.reshape(-1,1))
  train_y = scaler_y.transform(train_y.reshape(-1,1))
  valid_y = scaler_y.transform(valid_y.reshape(-1,1))
  test_y = scaler_y.transform(test_y.reshape(-1,1))

  train_X = train_X.reshape((train_X.shape[0], timestep, n_features))
  valid_X = valid_X.reshape((valid_X.shape[0], timestep, n_features))
  test_X = test_X.reshape((test_X.shape[0], timestep, n_features))
  return train_X, train_y, valid_X, valid_y, test_X, test_y

In [6]:
def make_data2(data, timestep, resid_check=False):
  values = data.values
  values = values.astype('float32')
  #scaler = MinMaxScaler(feature_range=(0, 1))
  #scaled = scaler.fit_transform(values)
  
  timestep = timestep
  n_features = data.shape[1]
  n_obs = timestep * n_features
  reframed = series_to_supervised(values, timestep, 1)
  reframed = reframed.iloc[: , :1-n_features]

  values = reframed.values
  indice1 = train_h.shape[0]
  indice2 = valid_h.shape[0]

  train = values[:indice1, :]
  valid = values[indice1:indice1+indice2, :]
  test = values[indice1+indice2:, :]

  train_X, train_y = train[:, :-1], train[:, -1]
  valid_X, valid_y = valid[:, :-1], valid[:, -1]
  test_X, test_y = test[:, :-1], test[:, -1]

  if (resid_check==True):
    train_y = resid_train.values[:quarto*2]
    valid_y = resid_train.values[quarto*2:]

  scaler = MinMaxScaler(feature_range=(0, 1)).fit(train_X)
  train_X = scaler.transform(train_X)
  valid_X = scaler.transform(valid_X)
  test_X = scaler.transform(test_X)

  scaler_y = MinMaxScaler(feature_range=(0, 1)).fit(train_y.reshape(-1,1))
  train_y = scaler_y.transform(train_y.reshape(-1,1))
  valid_y = scaler_y.transform(valid_y.reshape(-1,1))
  test_y = scaler_y.transform(test_y.reshape(-1,1))

  train_X = train_X.reshape((train_X.shape[0], timestep, n_features))
  valid_X = valid_X.reshape((valid_X.shape[0], timestep, n_features))
  test_X = test_X.reshape((test_X.shape[0], timestep, n_features))
  return train_X, train_y, valid_X, valid_y, test_X, test_y, scaler, scaler_y

# Test

# RNN

In [7]:
rmse_list = []
mae_list = []
mape_list = []

timestep=5
layers=1
num_units=128
dropout=0.1
lr=0.01
batch_size=32

for z in tqdm(range(10)):
  train_X, train_y, valid_X, valid_y, test_X, test_y, scaler, scaler_y = make_data2(hourly, 6, resid_check=False)
  model = Sequential()

  if layers > 1:
    model.add(SimpleRNN(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout, return_sequences=True))

    for i in range(layers-2):
      model.add(SimpleRNN(units = num_units, dropout=dropout, return_sequences=True))

    model.add(SimpleRNN(units = num_units, dropout=dropout))

  else:
    model.add(SimpleRNN(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout))

  model.add(Dense(units = 1))

  model.compile(
    loss="mse",
    optimizer=Adam(learning_rate=lr)
  )

  model.fit(train_X, train_y, batch_size=batch_size,
          epochs=200, verbose=0, shuffle=False,
          validation_data=(valid_X, valid_y),
          callbacks=[EarlyStopping(patience=10, restore_best_weights=True)])
  
  # make a prediction
  yhat = model.predict(test_X)
  yhat_inv = scaler_y.inverse_transform(yhat)
  #resid_sum = (yhat_inv+resid_test.values[3:])

  rmse_list.append(sqrt(mean_squared_error(yhat_inv, scaler_y.inverse_transform(test_y))))
  mae_list.append(mean_absolute_error(yhat_inv, scaler_y.inverse_transform(test_y)))
  mape_list.append(mean_absolute_percentage_error(yhat_inv, scaler_y.inverse_transform(test_y)))
  print()
print(f"RNN LIST OF RMSE: {rmse_list}")
print(f'RNN RMSE:  {sum(rmse_list)/len(rmse_list)}')

print(f"RNN LIST OF MAE: {mae_list}")
print(f'RNN MAE:  {sum(mae_list)/len(mae_list)}')

print(f"RNN LIST OF MAPE: {mape_list}")
print(f'RNN MAPE:  {sum(mape_list)/len(mape_list)}')

 10%|█         | 1/10 [00:04<00:42,  4.71s/it]

 20%|██        | 2/10 [00:08<00:33,  4.17s/it]

 30%|███       | 3/10 [00:14<00:35,  5.14s/it]

 40%|████      | 4/10 [00:18<00:26,  4.46s/it]

 50%|█████     | 5/10 [00:21<00:20,  4.02s/it]

...

 70%|███████   | 7/10 [00:31<00:13,  4.45s/it]

 80%|████████  | 8/10 [00:34<00:08,  4.06s/it]

 90%|█████████ | 9/10 [00:38<00:03,  3.97s/it]

100%|██████████| 10/10 [00:41<00:00,  4.15s/it]


RNN LIST OF RMSE: [0.7003577135803072, 0.6865878122461965, 0.7350159952148938, 0.7259039406608112, 0.6840407081991221, 0.6810548117757584, 0.7021280850788705, 0.7446157785728053, 0.7084407350982045, 0.7416116990568848]
RNN RMSE:  0.7109757279483854
RNN LIST OF MAE: [0.5439818, 0.52814007, 0.55447, 0.5702471, 0.51324546, 0.51443946, 0.5374672, 0.56139594, 0.53814703, 0.5699665]
RNN MAE:  0.5431500554084778
RNN LIST OF MAPE: [0.19444776, 0.194544, 0.23165487, 0.2028542, 0.20964135, 0.19123039, 0.19582798, 0.20988613, 0.19791694, 0.20368053]
RNN MAPE:  0.20316841453313828


# LSTM

In [8]:
rmse_list = []
mae_list = []
mape_list = []

timestep=3
layers=1
num_units=256
dropout=0
lr=0.01
batch_size=64

for z in tqdm(range(10)):
  train_X, train_y, valid_X, valid_y, test_X, test_y, scaler, scaler_y = make_data2(hourly, timestep, resid_check=False)
  model = Sequential()

  if layers > 1:
    model.add(LSTM(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout, return_sequences=True))

    for i in range(layers-2):
      model.add(LSTM(units = num_units, dropout=dropout, return_sequences=True))

    model.add(LSTM(units = num_units, dropout=dropout))

  else:
    model.add(LSTM(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout))

  model.add(Dense(units = 1))

  model.compile(
    loss="mse",
    optimizer=Adam(learning_rate=lr)
  )

  model.fit(train_X, train_y, batch_size=batch_size,
          epochs=200, verbose=0, shuffle=False,
          validation_data=(valid_X, valid_y),
          callbacks=[EarlyStopping(patience=10, restore_best_weights=True)])
  
  # make a prediction
  yhat = model.predict(test_X)
  yhat_inv = scaler_y.inverse_transform(yhat)
  #resid_sum = (yhat_inv+resid_test.values[3:])

  rmse_list.append(sqrt(mean_squared_error(yhat_inv, scaler_y.inverse_transform(test_y))))
  mae_list.append(mean_absolute_error(yhat_inv, scaler_y.inverse_transform(test_y)))
  mape_list.append(mean_absolute_percentage_error(yhat_inv, scaler_y.inverse_transform(test_y)))
  print()
print(f"LSTM LIST OF RMSE: {rmse_list}")
print(f'LSTM RMSE:  {sum(rmse_list)/len(rmse_list)}')

print(f"LSTM LIST OF MAE: {mae_list}")
print(f'LSTM MAE:  {sum(mae_list)/len(mae_list)}')

print(f"LSTM LIST OF MAPE: {mape_list}")
print(f'LSTM MAPE:  {sum(mape_list)/len(mape_list)}')

 10%|█         | 1/10 [00:08<01:14,  8.32s/it]

 20%|██        | 2/10 [00:13<00:52,  6.51s/it]

 30%|███       | 3/10 [00:18<00:41,  5.95s/it]

 40%|████      | 4/10 [00:26<00:40,  6.81s/it]

 50%|█████     | 5/10 [00:34<00:36,  7.24s/it]

...

 70%|███████   | 7/10 [00:50<00:22,  7.65s/it]

 80%|████████  | 8/10 [00:58<00:15,  7.67s/it]

 90%|█████████ | 9/10 [01:03<00:06,  6.98s/it]

100%|██████████| 10/10 [01:10<00:00,  7.08s/it]


LSTM LIST OF RMSE: [0.7605595302067697, 0.7454316961593923, 0.7346152054797602, 0.7348605235101909, 0.6314389191624487, 0.7628014328281526, 0.6536766414803654, 0.7354865893730375, 0.7406907096717927, 0.649762695349927]
LSTM RMSE:  0.7149323943221837
LSTM LIST OF MAE: [0.5945618, 0.5815748, 0.5750044, 0.5737687, 0.46701297, 0.5940406, 0.4842424, 0.57369465, 0.5762544, 0.48351163]
LSTM MAE:  0.5503666341304779
LSTM LIST OF MAPE: [0.22475283, 0.21925092, 0.21008869, 0.21311879, 0.17902598, 0.22112305, 0.20590444, 0.21521372, 0.21732926, 0.19995244]
LSTM MAPE:  0.21057601124048234


# GRU

In [10]:
rmse_list = []
mae_list = []
mape_list = []

timestep=12
layers=1
num_units=128
dropout=0.1
lr=0.01
batch_size=64

for z in tqdm(range(10)):
  train_X, train_y, valid_X, valid_y, test_X, test_y, scaler, scaler_y = make_data2(hourly, timestep, resid_check=False)
  model = Sequential()

  if layers > 1:
    model.add(GRU(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout, return_sequences=True))

    for i in range(layers-2):
      model.add(GRU(units = num_units, dropout=dropout, return_sequences=True))

    model.add(GRU(units = num_units, dropout=dropout))

  else:
    model.add(GRU(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout))

  model.add(Dense(units = 1))

  model.compile(
    loss="mse",
    optimizer=Adam(learning_rate=lr)
  )

  model.fit(train_X, train_y, batch_size=batch_size,
          epochs=200, verbose=0, shuffle=False,
          validation_data=(valid_X, valid_y),
          callbacks=[EarlyStopping(patience=10, restore_best_weights=True)])
  
  # make a prediction
  yhat = model.predict(test_X)
  yhat_inv = scaler_y.inverse_transform(yhat)
  #resid_sum = (yhat_inv+resid_test.values[3:])

  rmse_list.append(sqrt(mean_squared_error(yhat_inv, scaler_y.inverse_transform(test_y))))
  mae_list.append(mean_absolute_error(yhat_inv, scaler_y.inverse_transform(test_y)))
  mape_list.append(mean_absolute_percentage_error(yhat_inv, scaler_y.inverse_transform(test_y)))
print(f"GRU LIST OF RMSE: {rmse_list}")
print(f'GRU RMSE:  {sum(rmse_list)/len(rmse_list)}')

print(f"GRU LIST OF MAE: {mae_list}")
print(f'GRU MAE:  {sum(mae_list)/len(mae_list)}')

print(f"GRU LIST OF MAPE: {mape_list}")
print(f'GRU MAPE:  {sum(mape_list)/len(mape_list)}')

100%|██████████| 10/10 [01:12<00:00,  7.27s/it]

GRU LIST OF RMSE: [0.6538790835043569, 0.7020844070886689, 0.6890127272157085, 0.6889244408389368, 0.754932318047425, 0.652009044372396, 0.6806495697799149, 0.6870141696953993, 0.6384025962650716, 0.65361408647949]
GRU RMSE:  0.6800522443287368
GRU LIST OF MAE: [0.49601683, 0.52091444, 0.5163103, 0.51691467, 0.564438, 0.49293265, 0.5187793, 0.51735765, 0.4794026, 0.494641]
GRU MAE:  0.511770737171173
GRU LIST OF MAPE: [0.18771555, 0.21403438, 0.20255671, 0.20246017, 0.23699723, 0.18767487, 0.19576979, 0.20168878, 0.18109241, 0.18782921]
GRU MAPE:  0.19978190958499908


# Transformer

In [11]:
from tensorflow.keras import layers

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, feat_dim, num_heads, ff_dim, rate = 0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads = num_heads, key_dim = embed_dim)
        self.ffn = keras.Sequential( [layers.Dense(ff_dim, activation = "gelu"), layers.Dense(feat_dim),] )
        self.layernorm1 = layers.BatchNormalization()
        self.layernorm2 = layers.BatchNormalization()
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.embed_dim = embed_dim
        self.feat_dim = feat_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training = training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training = training)
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):

        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'feat_dim': self.feat_dim,
            'num_heads': self.num_heads,
            'ff_dim': self.ff_dim,
            'rate': self.rate,
        })
        return config

In [12]:
class Time2Vec(keras.layers.Layer):
    def __init__(self, kernel_size = 1):
        super(Time2Vec, self).__init__(trainable = True, name = 'Time2VecLayer')
        self.k = kernel_size

    def build(self, input_shape):
        # trend
        self.wb = self.add_weight(name = 'wb', shape = (input_shape[1],), initializer = 'uniform', trainable = True)
        self.bb = self.add_weight(name = 'bb', shape = (input_shape[1],), initializer = 'uniform', trainable = True)
        # periodic
        self.wa = self.add_weight(name = 'wa', shape = (1, input_shape[1], self.k), initializer = 'uniform', trainable = True)
        self.ba = self.add_weight(name = 'ba', shape = (1, input_shape[1], self.k), initializer = 'uniform', trainable = True)
        super(Time2Vec, self).build(input_shape)

    def call(self, inputs, **kwargs):
        bias = self.wb * inputs + self.bb
        dp = K.dot(inputs, self.wa) + self.ba
        wgts = K.sin(dp) # or K.cos(.)
        ret = K.concatenate([K.expand_dims(bias, -1), wgts], -1)
        ret = K.reshape(ret, (-1, inputs.shape[1] * (self.k + 1)))
        return ret

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1] * (self.k + 1))

    def get_config(self):

        config = super().get_config()
        config.update({
            'kernel_size': self.k,
        })
        return config

In [13]:
EPOCHS = 50
N_HEADS = 8
N_FOLDS = 10
FF_DIM = 256
N_BLOCKS = 6
EMBED_DIM = 64
BATCH_SIZE = 16
WINDOW_SIZE = 65
DROPUT_RATE = 0.0
TIME_2_VEC_DIM = 3
TRAIN_MODEL = True
SKIP_CONNECTION_STRENGTH = 0.9

In [14]:
rmse_list = []
mae_list = []
mape_list = []

batch_size=32
lr=0.001
N_HEADS = 8
FF_DIM = 256
N_BLOCKS = 8
EMBED_DIM = 32
DROPUT_RATE = 0.3
time2vec_dim = 4
timestep = 7

for z in tqdm(range(10)):
  train_X, train_y, valid_X, valid_y, test_X, test_y, scaler, scaler_y = make_data2(hourly, timestep, resid_check=False)


  input_shape = train_X.shape[1:]
  inp = Input(input_shape)
  x = inp

  time_embedding = keras.layers.TimeDistributed(Time2Vec(time2vec_dim - 1))(x)
  x = Concatenate(axis = -1)([x, time_embedding])
  x = layers.LayerNormalization(epsilon = 1e-6)(x)

  for k in range(N_BLOCKS):
    x_old = x
    transformer_block = TransformerBlock(EMBED_DIM, input_shape[-1] + ( input_shape[-1] * time2vec_dim), N_HEADS, FF_DIM, DROPUT_RATE)
    x = transformer_block(x)
    x = ((1.0 - SKIP_CONNECTION_STRENGTH) * x) + (SKIP_CONNECTION_STRENGTH * x_old)

  x = layers.Flatten()(x)

  x = layers.Dense(128, activation = "relu")(x)
  x = layers.Dropout(DROPUT_RATE)(x)
  x = Dense(1, activation = 'linear')(x)

  out = x
  model = Model(inp, out)

  model.compile(
    loss="mse",
    optimizer=Adam(learning_rate=lr)
              )

  model.fit(train_X, train_y, batch_size=batch_size,
          epochs=200, verbose=0, shuffle=False,
          validation_data=(valid_X, valid_y),
          callbacks=[EarlyStopping(patience=10, restore_best_weights=True)]
          )
  
  # make a prediction
  yhat = model.predict(test_X)
  yhat_inv = scaler_y.inverse_transform(yhat)
  #resid_sum = (yhat_inv+resid_test.values[3:])

  rmse_list.append(sqrt(mean_squared_error(yhat_inv, scaler_y.inverse_transform(test_y))))
  mae_list.append(mean_absolute_error(yhat_inv, scaler_y.inverse_transform(test_y)))
  mape_list.append(mean_absolute_percentage_error(yhat_inv, scaler_y.inverse_transform(test_y)))
print(f"Transformer LIST OF RMSE: {rmse_list}")
print(f'Transformer RMSE:  {sum(rmse_list)/len(rmse_list)}')

print(f"Transformer LIST OF MAE: {mae_list}")
print(f'Transformer MAE:  {sum(mae_list)/len(mae_list)}')

print(f"Transformer LIST OF MAPE: {mape_list}")
print(f'Transformer MAPE:  {sum(mape_list)/len(mape_list)}')

100%|██████████| 10/10 [11:14<00:00, 67.42s/it]

Transformer LIST OF RMSE: [0.8699763020214683, 0.8413737355416703, 0.7954657191569297, 1.4603522449311717, 0.8118474614125237, 0.9841744960459489, 0.9366877534015465, 0.9012484157088028, 0.7480001808513076, 1.17491102490481]
Transformer RMSE:  0.952403733397618
Transformer LIST OF MAE: [0.65538, 0.6505493, 0.6062527, 1.1582828, 0.6160899, 0.7662306, 0.7045543, 0.70393693, 0.5753491, 0.76571196]
Transformer MAE:  0.7202337503433227
Transformer LIST OF MAPE: [0.21512425, 0.25761938, 0.23177643, 0.32690662, 0.3446518, 0.2430423, 0.22583766, 0.52349263, 0.20998694, 0.9320982]
Transformer MAPE:  0.3510536223649979
